In [1]:
from hydradx.model.processing import get_omnipool_data, get_binance_orderbook, get_kraken_orderbook, get_omnipool, load_omnipool, save_omnipool
from hydradx.model.amm.omnipool_amm import OmnipoolState
from hydradx.model.amm.centralized_market import CentralizedMarket
from hydradx.model.amm.agents import Agent

import matplotlib.pyplot as plt

from pprint import pprint
from mpmath import mpf, mp
mp.dps = 50

from hydradxapi import HydraDX


In [2]:

RPC="wss://rpc.hydradx.cloud"
            
# asset_list, asset_map, tokens, fees = get_omnipool_data(RPC, archive=False)



In [3]:
# pprint(asset_list)

In [4]:
get_live_markets = True

if get_live_markets:
    omnipool = get_omnipool()
else:
    omnipool = load_omnipool()
pprint(omnipool)

Omnipool: omnipool
********************************
tvl cap: inf
LRNA imbalance: 0.0
lrna fee:

    4-Pool: 0.0519%
    HDX: 0.05%
    USDT10: 0.05%
    WETH: 0.05%
    2-Pool: 0.05%
    GLMR: 0.05%
    BNC: 0.05%
    vDOT: 0.0509%
    CFG: 0.05%
    DOT: 0.05%
    PHA: 0.05%
    ZTG: 0.05%
    INTR: 0.05%
    ASTR: 0.05%

asset fee:

    4-Pool: 0.2511%
    HDX: 0.288%
    USDT10: 0.25%
    WETH: 0.25%
    2-Pool: 0.25%
    GLMR: 1.098%
    BNC: 2.2422%
    vDOT: 0.25%
    CFG: 0.25%
    DOT: 0.2778%
    PHA: 0.25%
    ZTG: 0.25%
    INTR: 0.25%
    ASTR: 0.3388%

asset pools: (

    *4-Pool*
    asset quantity: 1553832.3831947728
    lrna quantity: 40362.74760875586
    USD price: 1.005508322882
    tvl: $1562391.3936663629
    weight: 40362.74760875586/533570.1750774072 (0.0756465587734552)
    weight cap: 1
    total shares: 1553832.3831947728
    protocol shares: 1553832.3831947728

    *HDX*
    asset quantity: 48975980.33937926
    lrna quantity: 44044.18269588679
    USD price:

In [5]:
# save = True
# if save:
#     save_omnipool(omnipool)

In [6]:
cex_asset_tkn = "INTR"
cex_denom_tkn = "USD"
dex_asset_tkn = "INTR"
dex_denom_tkn = "USDT"
one_hop = False
try:
    kraken_ob = get_kraken_orderbook((cex_asset_tkn, cex_denom_tkn), archive=False)
    cex = CentralizedMarket(
        order_book={(cex_asset_tkn, cex_denom_tkn): kraken_ob},
        asset_list=[cex_asset_tkn, cex_denom_tkn],
        trade_fee=0.001
    )
    one_hop = True
except:  # orderbook doesn't exist? try going through USD
    asset_ob = get_kraken_orderbook((cex_asset_tkn, "USD"), archive=False)
    denom_ob = get_kraken_orderbook((cex_denom_tkn, "USD"), archive=False)
    cex = CentralizedMarket(
        order_book={(cex_asset_tkn, "USD"): asset_ob, (cex_denom_tkn, "USD"): denom_ob},
        asset_list=[cex_asset_tkn, cex_denom_tkn, "USD"],
        trade_fee=0.001
    )
    

In [7]:
if one_hop:
    cex_spot = (cex.order_book[(cex_asset_tkn, cex_denom_tkn)].bids[0][0] + cex.order_book[(cex_asset_tkn, cex_denom_tkn)].asks[0][0]) / 2
else:
    cex_asset_spot = (cex.order_book[(cex_asset_tkn, "USD")].bids[0][0] + cex.order_book[(cex_asset_tkn, "USD")].asks[0][0]) / 2
    cex_denom_spot = (cex.order_book[(cex_denom_tkn, "USD")].bids[0][0] + cex.order_book[(cex_denom_tkn, "USD")].asks[0][0]) / 2
    cex_spot = cex_asset_spot / cex_denom_spot


# dex_denom_data = tokens[dex_denom_tkn]
# dex_denom_scale = mpf(1)
# overall_scale = mpf(1)
# dex_denom_lrna = dex_denom_data['LRNA'] * dex_denom_scale * overall_scale
# dot_liquidity = tokens[dex_asset_tkn]['liquidity'] * overall_scale
# dot_lrna = tokens[dex_asset_tkn]['LRNA'] * overall_scale
# dex_denom_liquidity = cex_spot * dex_denom_lrna * dot_liquidity / dot_lrna
# dex_denom_data_up = {'LRNA': dex_denom_lrna, 'liquidity': dex_denom_liquidity}
# hdx_data = {'LRNA': tokens['HDX']['LRNA'] * overall_scale, 'liquidity': tokens['HDX']['liquidity'] * overall_scale}
# dot_data = {'LRNA': dot_lrna, 'liquidity': dot_liquidity}

# op_state = OmnipoolState(
#     # tokens={dex_denom_tkn: dex_denom_data_up, dex_asset_tkn: dot_data, "HDX": hdx_data,
#     #         "USD": {"LRNA": mpf(10000), "liquidity": mpf(10000)}},
#     tokens=tokens,
#     preferred_stablecoin="USDT",
#     lrna_fee=0.0000,
#     asset_fee=0.0000
# )
op_state = omnipool

# protocol_agent = Agent(holdings={"DOT": mpf(500000)})
# op_state.add_liquidity(protocol_agent, 500000, "DOT")

protocol_agent = Agent(holdings={"GLMR": mpf(600000)})
op_state.add_liquidity(protocol_agent, 600000, "GLMR")


# op_state.asset_fee = 0
# op_state.lrna_fee = 0

    
# pprint(tokens)
print(op_state.liquidity)

{'4-Pool': 1553832.3831947728, 'HDX': 48975980.33937926, 'USDT10': 2707740.702887, 'WETH': 1112.6027491264585, '2-Pool': 54.1593593293248, 'GLMR': 1253838.7184088551, 'BNC': 575968.7671081127, 'vDOT': 229724.160459049, 'CFG': 1028763.3564301317, 'DOT': 733721.7435869717, 'PHA': 1169613.1951447497, 'ZTG': 2097955.5692162756, 'INTR': 14355418.918363446, 'ASTR': 2755680.9221255924}


In [8]:
op_spot = op_state.price(op_state, dex_asset_tkn, dex_denom_tkn)
pprint(op_spot)
pprint(cex_spot)
# pprint((cex.order_book[(cex_asset_tkn, cex_denom_tkn)].bids[0][0],cex.order_book[(cex_asset_tkn, cex_denom_tkn)].asks[0][0]))
# pprint((cex.order_book[(cex_asset_tkn, cex_denom_tkn)].asks[0][0] - cex.order_book[(cex_asset_tkn, cex_denom_tkn)].bids[0][0])/cex.order_book[(cex_asset_tkn, cex_denom_tkn)].bids[0][0])

0
0.07205


In [9]:
holdings = {}
for tkn in [cex_asset_tkn, cex_denom_tkn, dex_asset_tkn, dex_denom_tkn, "USD"]:
    if tkn not in holdings:
        holdings[tkn] = mpf(10000000)
agent = Agent(holdings=holdings)

asset_spot = omnipool.price(omnipool, dex_denom_tkn, "USDT10")
dollar_step_size = 2
trade_size_list = [dollar_step_size/asset_spot*i for i in range(1, 1500)]
dot_out_cex = []
dot_out_dex = []
for trade_size in trade_size_list:
    temp_op_state = op_state.copy()
    temp_cex_state = cex.copy()
    temp_agent1 = agent.copy()
    temp_agent2 = agent.copy()
    temp_op_state.swap(temp_agent1, tkn_buy=dex_asset_tkn, tkn_sell=dex_denom_tkn, sell_quantity=trade_size)
    if one_hop:
        temp_cex_state.swap(temp_agent2, tkn_buy=cex_asset_tkn, tkn_sell=cex_denom_tkn, sell_quantity=trade_size)
    else:
        temp_cex_state.swap(temp_agent2, tkn_buy="USD", tkn_sell=cex_denom_tkn, sell_quantity=trade_size)
        sell_amt_usd = temp_agent2.holdings["USD"] - agent.holdings["USD"]
        temp_cex_state.swap(temp_agent2, tkn_buy=cex_asset_tkn, tkn_sell="USD", sell_quantity=sell_amt_usd)
    dot_out_dex.append(temp_agent1.holdings[dex_asset_tkn] - agent.holdings[dex_asset_tkn])
    dot_out_cex.append(temp_agent2.holdings[cex_asset_tkn] - agent.holdings[cex_asset_tkn])
    # pprint(trade_size)
    # pprint(temp_op_state.price(temp_op_state, "DOT", "USDT"))
    
pprint("DEX")
plt.plot(trade_size_list, dot_out_dex, label="DEX")
# pprint(dot_out_dex)

pprint("CEX")
# pprint(dot_out_cex)
plt.plot(trade_size_list, dot_out_cex, label="CEX")
plt.legend()
plt.show()

diffs = [dot_out_dex[i] - dot_out_cex[i] for i in range(len(dot_out_dex))]
plt.plot(trade_size_list, diffs, label="DEX - CEX")
plt.show()

print(cex.order_book)

ZeroDivisionError: division by zero

In [ ]:
print(dot_out_cex[-1])

In [ ]:
print(dot_out_dex[-1])